In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

In [36]:
class Generator(nn.Module): #! nn.Module hội tụ đủ thứ của nn
    def __init__(self):
        super(Generator,self).__init__() #! inheritance from nn.Module
        #! generator architecture
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=64,kernel_size=4,stride=2,padding=1)
        self.conv2 = nn.Conv2d(in_channels=64,out_channels=128,kernel_size=4,stride=2,padding=1)
        self.conv3 = nn.Conv2d(in_channels=128,out_channels=256,kernel_size=4,stride=2,padding=1)
        self.deconv1 = nn.ConvTranspose2d(256,128,kernel_size=4,stride=2,padding=1)
        self.deconv2 = nn.ConvTranspose2d(128,64,kernel_size=4,stride=2,padding=1)
        self.deconv3 = nn.ConvTranspose2d(64,out_channels=1,kernel_size=4,stride=2,padding=1)
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
    def forward(self,x):
        #! pass input through:
        # Pass the input through the generator network
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.relu(x)
        x = self.deconv1(x)
        x = self.relu(x)
        x = self.deconv2(x)
        x = self.relu(x)
        x = self.tanh(self.deconv3(x))
        return x
        
    

In [37]:
class Discriminator(nn.Module):
    def __init__(self, *args, **kwargs):
        super(Discriminator,self).__init__()
        self.conv1 = nn.Conv2d(1,64,4,2,1)
        self.conv2 = nn.Conv2d(64,128,4,2,1)
        self.conv3 = nn.Conv2d(128,256,4,2,1)
        self.fc1 = nn.Linear(256*4*4,1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    def forward(self,x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.relu(x)
        x = x.view(-1, 256 * 4 * 4)
        x = self.fc1(x)
        x = self.sigmoid(x)
        return x
    

In [41]:
# Create the generator and discriminator models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
generator = Generator()
discriminator = Discriminator()

# Define the loss functions
generator_loss = nn.BCELoss()
discriminator_loss = nn.BCELoss()

# Create the optimizers for the generator and discriminator
generator_optimizer = optim.Adam(generator.parameters())
discriminator_optimizer = optim.Adam(discriminator.parameters())

# Load the training data
# ...
num_epochs = 2
# 
import torch
from torch.utils.data import DataLoader


# Define the transform to apply to the images
#! It allows for chaining multiple transformations together to create a more complex preprocessing pipeline. transforms.compose
#! This transformation converts the input image data from NumPy arrays to PyTorch tensors: transforms.ToTensor()
#! transforms.normalize(): 
#! This transformation normalizes the pixel values of the input images. 
# It subtracts the mean value (0.5) from each pixel and divides by the standard deviation (0.5). 
# This normalization helps to center the data and improve the performance of deep learning models.
transform = transforms.Compose([transforms.ToTensor(),
  transforms.Normalize((0.5,), (0.5,))
])

# Load the training data
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)

# Create a data loader
train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=0,
)
# Train the GAN model
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(train_loader):
        # Generate fake images
        fake_images = generator(images)

        # Train the discriminator on real and fake images
        real_labels = torch.ones(images.shape[0]) #! ma trận toàn 1, dimension đầu tiên của image (theo hàng)
        fake_labels = torch.zeros(fake_images.shape[0])

        real_output = discriminator(images)
        discriminator_real_loss = discriminator_loss(real_output, real_labels)

        fake_output = discriminator(fake_images)
        discriminator_fake_loss = discriminator_loss(fake_output, fake_labels)

        discriminator_total_loss = discriminator_real_loss + discriminator_fake_loss

        # Train the generator on the discriminator's output

ValueError: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([18, 1])) is deprecated. Please ensure they have the same size.